### Link to Google Drive: https://drive.google.com/drive/folders/1Pnuo1tB1XtiDjMa7eXmuUctL2XX9emU7
### Link to Tableau Online: https://dub01.online.tableau.com/#/site/telenordashboard/projects/123532
### Link to Google Cloud Storage: https://console.cloud.google.com/storage/browser?project=telenor-data-science
### Link to Google Cloud BigQuery: https://console.cloud.google.com/bigquery?project=telenor-data-science&p=telenor-data-science&d=telenor_dataset&page=dataset

In [1]:
import glob
import os
import pandas as pd

In [2]:
# Project info
PROJECT_ID = 'telenor-data-science'
DATASET_DIR = 'location_dataset'

In [3]:
# Download dataset from GCS
!mkdir -p data
!gsutil -m cp -r gs://{PROJECT_ID}/datasets/{DATASET_DIR} /content/data

Copying gs://telenor-data-science/datasets/location_dataset/bekkestua.parquet...
Copying gs://telenor-data-science/datasets/location_dataset/all_locations.parquet...
Copying gs://telenor-data-science/datasets/location_dataset/bygdøy_alle.parquet...
Copying gs://telenor-data-science/datasets/location_dataset/alnabru.parquet...
Copying gs://telenor-data-science/datasets/location_dataset/bryn_skole.parquet...
Copying gs://telenor-data-science/datasets/location_dataset/alvim.parquet...
Copying gs://telenor-data-science/datasets/location_dataset/danmarks_plass.parquet...
Copying gs://telenor-data-science/datasets/location_dataset/bakke_kirke.parquet...
Copying gs://telenor-data-science/datasets/location_dataset/bankplassen.parquet...
Copying gs://telenor-data-science/datasets/location_dataset/e6_alna_senter.parquet...
Copying gs://telenor-data-science/datasets/location_dataset/e6_tiller.parquet...
Copying gs://telenor-data-science/datasets/location_dataset/eilif_dues_vei.parquet...
Copying 

In [ ]:
# Upload data as BigQuery tables
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {PROJECT_ID}

# List all files in storage
from google.cloud import storage
storage_client = storage.Client(PROJECT_ID)
blobs = list(storage_client.list_blobs(PROJECT_ID, prefix=f'datasets/{DATASET_DIR}/'))
paths = [f"gs://{PROJECT_ID}/{blob.name}" for blob in blobs]
filenames = [os.path.basename(blob.name).split('.')[0] for blob in blobs]
print(paths)
print(filenames)


Updated property [core/project].
['gs://telenor-data-science/datasets/location_dataset.tar.gz', 'gs://telenor-data-science/datasets/location_dataset/alnabru.parquet', 'gs://telenor-data-science/datasets/location_dataset/alvim.parquet', 'gs://telenor-data-science/datasets/location_dataset/bakke_kirke.parquet', 'gs://telenor-data-science/datasets/location_dataset/bankplassen.parquet', 'gs://telenor-data-science/datasets/location_dataset/bekkestua.parquet', 'gs://telenor-data-science/datasets/location_dataset/bryn_skole.parquet', 'gs://telenor-data-science/datasets/location_dataset/bygdøy_alle.parquet', 'gs://telenor-data-science/datasets/location_dataset/danmarks_plass.parquet', 'gs://telenor-data-science/datasets/location_dataset/e6_alna_senter.parquet', 'gs://telenor-data-science/datasets/location_dataset/e6_tiller.parquet', 'gs://telenor-data-science/datasets/location_dataset/eilif_dues_vei.parquet', 'gs://telenor-data-science/datasets/location_dataset/elgeseter.parquet', 'gs://teleno

In [ ]:
# Create new BigQuery dataset
BIGQUERY_DATASET_NAME = "location_dataset"
!bq mk --location europe-west1 --dataset {BIGQUERY_DATASET_NAME}

# Create tables
for path, filename in list(zip(paths, filenames)):
    !bq load  --source_format=PARQUET --autodetect {BIGQUERY_DATASET_NAME}.{filename} {path} 

Dataset 'telenor-data-science:location_dataset' successfully created.
Waiting on bqjob_r239e7eee4846c343_000001753316f913_1 ... (0s) Current status: DONE   
BigQuery error in load operation: Error processing job 'telenor-data-
science:bqjob_r239e7eee4846c343_000001753316f913_1': Error while reading data,
error message: Input file is not in Parquet format.
Waiting on bqjob_rb9e42401745bda0_00000175331708d0_1 ... (14s) Current status: DONE   
Waiting on bqjob_r4702c6f4f12dba68_0000017533174f3c_1 ... (7s) Current status: DONE   
Waiting on bqjob_r250107866f83ca2d_00000175331778ae_1 ... (9s) Current status: DONE   
Waiting on bqjob_r5daf7921f738ebf_000001753317aaaa_1 ... (9s) Current status: DONE   
Waiting on bqjob_r112753810417c63_000001753317ddaa_1 ... (9s) Current status: DONE   
Waiting on bqjob_r62c8bdead04f20ec_0000017533181087_1 ... (6s) Current status: DONE   
Waiting on bqjob_r7a3542c92d6f425b_000001753318377c_1 ... (8s) Current status: DONE   
Waiting on bqjob_r22a1b325323e82d1_

In [ ]:
for path, filename in list(zip(paths, filenames))[10:]:
    if ',' in path:
        path = f'"{path}"'
        filename = filename.replace(',', '_')
        !bq load  --source_format=PARQUET --autodetect {BIGQUERY_DATASET_NAME}.{filename} {path}

BigQuery error in load operation: All URIs must begin with "gs://" if any do.
BigQuery error in load operation: All URIs must begin with "gs://" if any do.
BigQuery error in load operation: All URIs must begin with "gs://" if any do.
BigQuery error in load operation: All URIs must begin with "gs://" if any do.


In [ ]:
# Create new BigQuery dataset
BIGQUERY_DATASET_NAME = "location-dataset"
!bq mk --location europe-west1 --dataset {BIGQUERY_DATASET_NAME}

# Create new table for every parquet file
for path in glob.glob(f"/content/data/{DATASET_DIR}/**/*.parquet", recursive=True)[:4]:
    table_name = os.path.basename(path).split('.')[0]
    print(table_name)
    
    !bq load  --source_format=PARQUET --autodetect {BIGQUERY_DATASET_NAME}.{table_name} gs://{PROJECT_ID}/datasets/{DATASET_DIR}/{table_name}.parquet
    
# This loads ALL ROWS into a single table ... :(
#!bq load  --source_format=PARQUET --autodetect {BIGQUERY_DATASET_NAME}.{table_name} gs://{PROJECT_ID}/datasets/{DATASET_DIR}/*.parquet

Updated property [core/project].
BigQuery error in mk operation: Invalid dataset ID "location-dataset". Dataset
IDs must be alphanumeric (plus underscores) and must be at most 1024 characters
long.
minnesundvegen,_gjøvik
BigQuery error in load operation: All URIs must begin with "gs://" if any do.
solheim
BigQuery error in load operation: Not found: Dataset telenor-data-
science:location-dataset
knarrdalstranda
BigQuery error in load operation: Not found: Dataset telenor-data-
science:location-dataset
lensmannsdalen
BigQuery error in load operation: Not found: Dataset telenor-data-
science:location-dataset


In [ ]:
!bq load --source_format=PARQUET --autodetect project:dataset:table gs://bucket/subdir/*/file.parquet

In [ ]:
if DATASET == DATASET_JOINED:
    !tar -zxf /tmp/{DATASET} -C /content/data
elif DATASET == DATASET_SEPARATE:
    !unzip /tmp/{DATASET} -d /content/data

    telenor-data-science/datasets/processed_dataset_joined



FROM = "/tmp/processed_dataset_joined"
TO = "gs://telenor-data-science/datasets/processed_dataset_joined"
IS_DIR = True

if IS_DIR:
    !gsutil -m cp -r {FROM} {TO}
else:
    !gsutil cp {FROM} {TO}

In [ ]:
!bq mk --help